In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [49]:
#Load the Dataset
data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
## Preprocess the data
### Drop irrelevant columns 
data = data.drop(['RowNumber','CustomerId','Surname'], axis = 1) #axis=1 means column wise
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [51]:
## Encode Categorical Variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [52]:
#One Hot Encode 'Geography'
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo  = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [53]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [54]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [55]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [56]:
## Combine one hot encoded columns with the original data
data = pd.concat([data.drop('Geography', axis=1), geo_encoded_df], axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [80]:
label_encoder_gender

LabelEncoder()

In [81]:
onehot_encoder_geo

OneHotEncoder()

In [57]:
## Save the encoders and the scalers
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)



In [58]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [59]:
## Divide the dataset into dependent and independent features
X = data.drop('Exited', axis = 1)
y = data['Exited']

## Split the data into training and testing 
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
# random state keeps the train and test datasets same each time. If it is not mentiones, that value of train and test data will be different each time. 

##Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)




In [60]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [61]:
with open('sscaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

In [62]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


## ANN Implementation 

In [63]:
'''## Tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import dense 
from tensorflow.keras.callbacks import EarlyStopping, Tensorboard
import datetime 



#Build ANN Model
model = Sequential([
     Dense(64,activation='relu', input_shape = (X_train.shape[1],)),##HL1 Connected with input layer
     Dense(32, activation='relu'), ##HL2
     Dense(1, activation='sigmoid')##output layer
]
)

model.summary()

##mention optimizer
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensrflow.keras.losses.BinaryCrossenntropy()

## compile model
model.compile(optimizer=opt, loss = "binary_crossentropy", metrics=['accuracy'] )

##Setup the Tensorboard
## We setup the tensorboard to capture the loss after the training 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

tensorflow_callback = TensorBoard(log_dir= log_dir, histogram_freq=1)

##Set up Early stopping --->whenever loss value stops decreasing by a significant amount, it stops training 
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True )

## Training the Model 
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks = [tensorflow_callback, early_stopping_callback]
)
##y_test data will be used for comparision with respect to the accuracy

##SAVE this Model file
model.save('model.h5')

## Load Tensorboard Extension 
%load_ext tensorboard

%tensorboard --logdir logs/fit
,,,


SyntaxError: incomplete input (1947893791.py, line 1)

## Import Libraries

In [64]:
## PyTorch COde
import torch
import torch.nn as nn
import torch.optim as optim 
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import datetime


## Define the Model

In [65]:
class ANNModel(nn.Module): #it inherits from nn.Module which is the base class for all models in PyTorch
    #init function is called when an object is instantiated. input size is passes to define the input dimensions of the model

    def __init__(self, input_size):
        super(ANNModel, self).__init__() #initializes the base class nn.Module and ensures all layers are registered correctly
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    #This method defines the forward pass of the model 
    #The input x passes through each layer and activation funciton in sequence 

    def forward(self, x):
        x = self.relu(self.fc1(x)) #input is passed through the first layer and then the ReLu Activarion function is applied

        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x # final output of the model
    
input_dim = X_train.shape[1]
model = ANNModel(input_dim)


## Prepare the Data for PyTorch

In [66]:
#converts data from a numpy array to a PyTorch tensor of type float.
#PyTorch model requires data in Tensor Format
X_train_tensor = torch.tensor(X_train).float()
X_test_tensor = torch.tensor(X_test).float()

# Converts target variable into tensors
y_train_tensor = torch.tensor(y_train.values).float()
y_test_tensor = torch.tensor(y_test.values).float()

#Creates Tensor Dataset for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

#The DataLoader loads the dataset in batches of 32 and shuffles the data during training 
train_loader = DataLoader(train_dataset, batch_size=32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle= False)

## Define Loss Function and Optimizer 

In [67]:
#Defines the loss function as Binary Cross Entropy Loss
loss_fn = nn.BCELoss()
# Defines the optimizer Adam and learning rate 0.01. This will update the model parameters during training
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [68]:
#Initializes the Tensorboard writer to log metrics during training.
#This is further used in visualization of model training.
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
writer = SummaryWriter(log_dir)
def calculate_accuracy(y_pred, y_true):
    predicted = (y_pred > 0.5).float()  # For binary classification
    correct = (predicted == y_true).float().sum()
    accuracy = correct / len(y_true)
    return accuracy


## Training Loop with Early Stopping

In [69]:
num_epochs = 100
patience = 5
best_val_loss = float('inf')
early_stop_count = 0


for epoch in range(num_epochs):
    model.train()   #puts model into training mode
    #this is important because some layers such as drop out and batch normalization behave differently during training 
    total_loss = 0
    total_acc = 0

    for X_batch, y_batch, in train_loader:
        #shape of X_batch: [batch_size, num_features]
        #shape of y_batch: [batch_size]
        optimizer.zero_grad()  #resets gradient of the model parameters to 0. This is important because gradient get accumulated by default in Pytorch, so we need to reset them after every batch
        
        #forward pass: X_batch is passed through thr model to get predictions
        # shape after forward pass: [batch_size,1], where each value represents the predicted possibility of the churn
        # .squeeze() changes to [batch_size]. Each element corresponds to probability of a sample in batch
        predictions = model(X_batch).squeeze() 

        # the bce compares the model's prediction with true labels
        loss = loss_fn(predictions, y_batch)
        accuracy = calculate_accuracy(predictions, y_batch)


        #backward pass: Computes the gradients of the loss wrt model's parameters. The gradients are stored in the parameter's .grad attribute
        loss.backward()

        #updates the model's parameters using computed gradients. This is where model "learns by adjusting the weights
        optimizer.step()

        total_loss += loss.item()
        total_acc+=accuracy.item()

    avg_train_loss = total_loss / len(train_loader)
    avg_train_acc = total_acc / len(train_loader)

    #Validation step
    model.eval()
    val_loss = 0
    val_acc = 0
    with torch.no_grad():
        for X_val, y_val in test_loader:
            val_predictions = model(X_val).squeeze()
            val_loss += loss_fn(val_predictions, y_val).item()

            val_acc = calculate_accuracy(val_predictions,y_val).item()
    

    avg_val_loss = val_loss / len(test_loader)
    avg_val_acc = val_acc / len(test_loader)

    #using tensorboard to log the training and validation losses for each epoch
    #total_loss/len(train_loader)--> total_loss accumulated over all batches in the epoch and divide it by number  of batches to get the average loss for the epoch
    #the third argument "epoch" is the x-axis value in Tensorboard.
    writer.add_scalar('Loss/train', total_loss/len(train_loader),epoch)
    writer.add_scalar('Accuracy/train', avg_train_acc, epoch)
    writer.add_scalar('Loss/val', val_loss/len(test_loader), epoch)
    #writer.add_scalar('Accuracy/train', train_accuracy, epoch)
    writer.add_scalar('Accuracy/val', avg_val_acc, epoch)


    #EARLY STOPPING
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stop_count = 0
        torch.save(model.state_dict(), 'best_model.pth')
    else:
        early_stop_count+=1
        if early_stop_count >=patience:
            print(f'Early Stopping at epoch {epoch}')
            break

Early Stopping at epoch 12


## Start TensorBoard

In [70]:
%load_ext tensorboard
%tensorboard --logdir logs/fit/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 729284), started 1 day, 18:39:29 ago. (Use '!kill 729284' to kill it.)

In [71]:

%reload_ext tensorboard

## Saving and Loading Model

In [72]:
#Save the model
torch.save(model.state_dict(), 'model.pth')
#Load the model
model.load_state_dict(torch.load('model.pth'))

C:\Users\tanya\AppData\Local\Temp\ipykernel_580488\772265562.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [73]:
## prediction with the model using tensorflow aand keras 
##Load the pickle file 


